In [18]:
import numpy as np
import os
os.system('clear')

import pygimli as pg
from pygimli.meshtools import appendTriangleBoundary, merge2Meshes
from pygimli.mplviewer import drawMesh
from pygimli.viewer import showMesh

In [44]:
#Ici on commence par définir un maillage rectangulaire homogène

xmin, xmax = 0., 50. #On définit les limites en x du maillage
zmin, zmax = -50., -25. #On définit les limites en z du maillage

dx = 1.0 #On définit le pas de la maille
xreg = np.arange(xmin, xmax + dx, dx, 'float') #On calcule les coordonées des mailles
zreg = np.arange(zmin, zmax + dx, dx, 'float')

mesh1 = pg.Mesh(2) #On crée un maillage vide 
mesh1.create2DGrid(xreg, zreg, 0) #On crée le maillage
for c in mesh1.cells(): #On marque les cellules
    c.setMarker(1)

print(mesh1) #On affiche le mombre de noeuds, cellules et lignes

#showMesh(mesh1, mesh1.cellMarkers(), logScale=False, label="Region marker") #On définit les paramètres d'affichage

showMesh(mesh1, mesh1.cellMarkers(), label="Region marker")
#drawMesh(ax,mesh1)

Mesh: Nodes: 1326 Cells: 1250 Boundaries: 2575
No valid data:  1 1 False


(<matplotlib.axes._subplots.AxesSubplot at 0x7ff483677630>, None)

In [35]:
#A présent on crée une zone grâce à un polygone

poly = pg.Mesh(2)  # empty 2d mesh
nStart = poly.createNode(xmin, zmax, 0.0) #On crée un noeud de départ, on travaille en 2D donc le dernier terme vaut 0.0

nA = nStart #On définit le noeud de départ
for x in xreg[1:]: #On démarre de 1 et on se balade sur l'axe des x en créant un noeud à chaque fois
    nB = poly.createNode(x, zmax, 0.0)
    poly.createEdge(nA, nB) #On définit un côté entre le noeud précédemment crée et le nouveau
    nA = nB #On remplace le noeud de départ par le noeud nouvellement crée

z2 = 0. #On définit une altitude z2 
nA = poly.createNode(xmax, z2, 0.0) #On crée un noeud
poly.createEdge(nB, nA) #On fait le lien entre le dernier noeud crée et celui-là
nB = poly.createNode(xmin, z2, 0.0) #On crée un autre noeud en symétrique
poly.createEdge(nA, nB) #On fait le lien avec le noeud précédent
poly.createEdge(nB, nStart) #On ferme le polygone!

tri = pg.TriangleWrapper(poly) #On appelle la fonction triangle
tri.setSwitches('-pzeAfaq31') #On rentre tout un tas de commandes rigolotes pour générer les triangles
# Ici on a :
# p : planar straight line graph ==> fichier poly
# z : on démarre le comptage à 0
# A : assigne un attribut à chaque triangle qui indique à quel segment il appartient et est lié
# f : algorithme de triangulation (?)
# a : impose une surface contrainte pour chaque triangle on peut ajouter un nombre si on veut préciser
# q31 : impose que les triangles générés aient au minimun des angles de 20° On peut ajouter un nombre derrière pour préciser le nombre que l'o souhaite

In [36]:
#A présent on génère le maillage hétérogène

mesh2 = pg.Mesh(2) #On appelle le second maillage autour du premier
tri.generate(mesh2) #On génère les triangles au sein du polygone précédemment crée

for cell in mesh2.cells(): #On génère les cellules de chaque maille
    cell.setMarker(2)

In [48]:
#Fusion des deux maillages pour créer un maillage hybride
mesh3 = merge2Meshes(mesh1, mesh2) #On peut faire fusionner les deux maillages ainsi crées en un seul

showMesh(mesh3, mesh1.cellMarkers(), label="Region marker")

No valid data:  1 1 False


(<matplotlib.axes._subplots.AxesSubplot at 0x7ff481d30470>, None)

In [50]:
#Autre aspect, on peut créer un maillage autour de celui déjà crée

# On crée un maillage de dimension 100*100 autour du maillage hybride, qualité de maillage équivalente, lissée, marquage
mesh = appendTriangleBoundary(mesh3, -100., 100., quality=31, smooth=True,
                              marker=3, isSubSurface=True)

#On affiche le maillage avec les marquages, avec cette technique il semble qu'on peut avoir l'option pour cliquer sur les mailles
ax, cbar = showMesh(mesh, mesh.cellMarkers(), cmap="summer",
                    label="Region marker")

drawMesh(ax, mesh)

ax, _ = showMesh(mesh, mesh.cellMarkers(), logScale=False,
                 label="Region marker")

drawMesh(ax, mesh)
pg.wait()

In [42]:
print(xreg[1:])

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.
  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.  45.
  46.  47.  48.  49.  50.]


In [43]:
print(zreg)

[-50. -49. -48. -47. -46. -45. -44. -43. -42. -41. -40. -39. -38. -37. -36.
 -35. -34. -33. -32. -31. -30. -29. -28. -27. -26. -25.]
